In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from utils import *

import tensorflow as tf

In [2]:
# This gives us the cemera perspective matrix
def getPerspectiveProjection(near, far, top, right):
    return np.array([
        [near/right, 0, 0, 0],
        [0, near/top, 0, 0],
        [0, 0, -(far+near)/(far-near), -2*far*near/(far-near)],
        [0, 0, -1, 0]
    ])

In [3]:
points = pd.read_csv('data/remeetingtoday/fullData.csv', header=None).T.to_numpy()

# We are not using the distance matrices from here as I do not know the projection matrix
D_1 = pd.read_csv('data/remeetingtoday/D_1.csv', header=None).T.to_numpy()
D_2 = pd.read_csv('data/remeetingtoday/D_2.csv', header=None).T.to_numpy()
D_3 = pd.read_csv('data/remeetingtoday/D_3.csv', header=None).T.to_numpy()
D_4 = pd.read_csv('data/remeetingtoday/D_4.csv', header=None).T.to_numpy()

In [4]:
def embed(data, projections=None, hidden_points_idx=None):
    # Calculate the distance matrix for the given points
    dist_mats = [distance_matrix(points, points) for points in data]
    
    # Generate (200,3) random uniform data, 
    # Meaning, generate 200 points randomly in the given point space
    min_val = data.min()
    max_val = data.max()
    X = np.random.uniform(min_val, max_val, (dist_mats[0].shape[0], 3))
    
    # Convert to homogeneous: (200,3) => (200,4)
    X = tf.Variable(to_homogeneous(X))
    
    # This is our stress function, we will minimuze this
    @tf.function
    def loss():
        total = []
        # For each projection matrix, we calculate the distance matrix of the projected points
        # Later, we will also use the projection matrices as variable and minimice the loss based on them
        for dist_mat, projection in zip(dist_mats, projections):
            # Dot project X with projection matrix to get the projected points
            projected = X @ projection
            
            # Calculate the distance matrix D
            # Taken from: https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow
            r = tf.reduce_sum(projected*projected, 1)
            r = tf.reshape(r, [-1, 1])
            D = r - 2 * (projected @ tf.transpose(projected)) + tf.transpose(r)
            
            # Collect Distance matrix for all the perspective projections
            total.append(dist_mat - D)
        
        # Sum all the distance matrix together to get single value
        return tf.reduce_sum(total)
    
    # Using tensorflow Stochastice Gradient Descent to minimize loss
    opt = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    # Running for 10 iterations
    for i in range(100000):
        if i % 1000 == 0:
            print("At iteration", i)
            print("Loss : ", loss().numpy())
        opt.minimize(loss, var_list=[X])
    
    # Converting from homogenious to normal point form: (200,4) => (200, 3)
    return (X.numpy()/X.numpy()[:, 3:4])[:, :3]

In [5]:
transformed_points = []
perspectives = []
for angle in [30, 60, 90]:
    perspectives.append(
        getPerspectiveProjection(0.1, 100, 1.5, 1.5).dot(getRotationMatrix(angle, "z"))
    )
    transformed_points.append(to_homogeneous(points).dot(perspectives[-1])[:, :3])
    
transformed_points = np.array(transformed_points)
perspectives = np.array(perspectives)

X = embed((transformed_points/transformed_points[:, :, 2:3])[:, :, :2], perspectives)

2021-09-29 02:26:04.680824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-29 02:26:04.680846: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-29 02:26:04.680861: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (insane-PC): /proc/driver/nvidia/version does not exist
2021-09-29 02:26:04.681016: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


At iteration 0
Loss :  3369.8027099434266


2021-09-29 02:26:04.912955: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


At iteration 1000
Loss :  nan
At iteration 2000
Loss :  nan


KeyboardInterrupt: 

In [ ]:
X.shape

In [ ]:
range_xyz = [X.min(), X.max()]

plot_3D([X], range_x=range_xyz, range_y=range_xyz, range_z=range_xyz).show()